In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas.tools.plotting import scatter_matrix

In [ ]:
weather = pd.read_csv('../input/weather.csv', low_memory=False)
trips = pd.read_csv('../input/trip.csv', low_memory=False)
stations = pd.read_csv('../input/station.csv', low_memory=False)
lunes, martes, miercoles, jueves, viernes, sabado, domingo = 0,1,2,3,4,5,6
dias = ["lunes", "martes", "miercoles", "jueves", "viernes", "sabado", "domingo"]
meses = ['Enero', 'Febrero', 'Marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre',
              'noviembre', 'diciembre']

In [ ]:
trips["date"] = pd.to_datetime(trips.start_date, format='%m/%d/%Y %H:%M')
trips["month"] = trips["date"].apply(lambda x: x.month)
trips["day"] = trips["date"].apply(lambda x: x.day)
trips['weekday'] = trips['date'].apply(lambda x: x.weekday())
trips["hour"] = trips["date"].apply(lambda x: x.hour)
trips["year"] = trips["date"].apply(lambda x: x.year)
trips["DATE"] = pd.to_datetime(trips[['year', 'month', 'day']], yearfirst=True)
trips["trips"] = trips["day"].apply(lambda x: 1)
trips["recorrido"] = trips["start_station_name"] + " to " + trips["end_station_name"]
trips.start_date = pd.to_datetime(trips.start_date)
trips.start_date = trips.start_date.apply(lambda x: x.date())

In [ ]:
labels = meses
index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
trips_month = trips["month"].value_counts(sort=False)
barras = trips_month.plot(kind="bar")
barras.set(title = "Cantidad de viajes por mes", xlabel="Meses", ylabel="cantidad de viajes")
plt.xticks(index,labels)
plt.show()

In [ ]:
week = [0, 0, 0, 0, 0, 0, 0]
for day in trips["date"]:
    week[day.weekday()] += 1
index = [0, 1, 2, 3, 4, 5, 6]
labels = dias
plt.title("Cantidad total de viajes por dia de la semana")
plt.xlabel("Dias de la semana")
plt.ylabel('Cantidad de viajes')
plt.bar(index, week, align="center")
plt.xticks(index, labels)
plt.show()

In [ ]:
tripsHora = trips.groupby("hour")
tripsHora.count()["start_date"].plot(kind="bar")
plt.title("Cantidad total de viajes por hora")
plt.xlabel("Hora del dia")
plt.ylabel('Cantidad de viajes')
plt.show()

In [ ]:
aux = {}
c = 0
for dia in dias:
    aux[dia] = trips[trips['weekday'] == c ].groupby('DATE').aggregate(sum).reset_index()['trips'][:104]
    c += 1
aux = pd.DataFrame.from_dict(aux, orient='columns')
aux.plot.box()
plt.title('Cantidad de viajes por dia de la semana')
plt.xlabel('Dias de la semana')
plt.ylabel('Cantidad de viajes')
plt.show()

In [ ]:
aux = {}
c = 0
horas = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '15', '16', '17',
             '18', '19', '20', '21','22','23']
for hora in horas:
    aux[hora] = trips[trips['hour'] == c].groupby('DATE').aggregate(sum).reset_index()['trips'][:104]
    c += 1
aux = pd.DataFrame.from_dict(aux, orient='columns')
aux.plot.box()
plt.title('Cantidad de viajes por hora del dia')
plt.xlabel('Dias de la semana')
plt.ylabel('Cantidad de viajes')
plt.show()

In [ ]:
Grafico: cantidad de viajes por cada dia del set

In [ ]:
trips_por_dia = trips.groupby('DATE').aggregate(sum)
trips_por_dia['trips'].plot()
plt.title('Cantidad de viajes por dia')
plt.xlabel('Dias')
plt.ylabel('Cantidad de viajes')
plt.show()

In [ ]:
trips3 = trips [["hour",'weekday','trips']]
trips2 = trips3.pivot_table(index = 'hour',columns = 'weekday',aggfunc=sum )
fig, ax = plt.subplots(figsize=(16,5))       # Sample figsize in inches
sns.heatmap(trips2,cmap='Oranges', xticklabels = dias)
sns.plt.show()

In [ ]:
Grafico heatmap: Las 10 estaciones de inicio mas frecuentes por hora

In [ ]:
def calcular_top_estaciones_inicio(cantidad_de_estaciones):
    cantidad_de_starts = trips[["start_station_name", "trips"]]
    cantidad_de_starts = cantidad_de_starts.groupby("start_station_name").count()
    ranking = cantidad_de_starts.sort_values(by="trips", ascending=False)[:cantidad_de_estaciones]
    return ranking
   
cantidad_de_estaciones=10
lista_top = []
top_estaciones = calcular_top_estaciones_inicio(cantidad_de_estaciones).reset_index()['start_station_name']
lista_top = []
top_estaciones.apply(lambda x: lista_top.append(x))
trips3 = trips[["hour", 'start_station_name', 'trips']]
trips3 = trips3[trips3.start_station_name.isin(lista_top)]
trips2 = trips3.pivot_table(index='hour', columns='start_station_name', aggfunc=sum)
sns.heatmap(trips2, cmap='Oranges')
sns.plt.show()

In [ ]:
cantidad_de_recorridos = trips[["recorrido","trips"]]
cantidad_de_recorridos = cantidad_de_recorridos.groupby("recorrido").count()
ranking_recorridos = cantidad_de_recorridos.sort_values(by = "trips", ascending=False)[:20]
ranking_recorridos.plot(kind="bar")
plt.title('Recorridos mas frecuentes')
plt.xlabel('Estaciones')
plt.ylabel('Cantidad de viajes')
plt.show()

In [ ]:
gruped = trips[["subscription_type","trips"]].groupby(["subscription_type"]).count()
gruped.plot(kind = "bar", y=["trips"])
plt.title("Cantidad total de viajes segun el tipo de subscripcion")
plt.show()

In [ ]:
plt.figure()
labels=["lunes", "martes", "miercoles", "jueves", "viernes", "sabado", "domingo"]	
trips.loc[trips.subscription_type.str.contains('Customer'),\
            ['trips','weekday']].groupby('weekday').count()\
            .loc[:,'trips'].plot(rot=90,xticks=range(0,7), \
            linewidth=2,figsize=(12,8),label='Customer');
trips.loc[trips.subscription_type.str.contains('Subscriber'),\
            ['trips','weekday']].groupby('weekday').count()\
            .loc[:,'trips'].plot(rot=90,xticks=range(0,7), \
            linewidth=2,label='Subscriber');
plt.legend();
plt.title("Cantidad total de viajes registrados por dia de semana")
plt.xticks(range(0,7), labels)
plt.show();

In [ ]:
trips[["bike_id","trips"]].groupby(["bike_id"]).count().sort_values(by = "trips", ascending=False)[:5].plot(kind = "bar")
plt.title("Top 5 bicicletas mas usadas")
plt.show()   

In [ ]:
trips[["bike_id","trips"]].groupby(["bike_id"]).count().sort_values(by = "trips", ascending=True)[:5].plot(kind = "bar")
plt.title("Top 5 bicicletas mas usadas")
plt.show()

In [ ]:
def func(x):
 if x == "Rain":
  return 2
 elif x == 'rain':
  return 2
 elif x == "Fog":
  return 1
 elif x == "Rain-Thunderstorm":
  return 0
 elif x == "Fog-Rain":
  return 3

weather["date"] = pd.to_datetime(weather.date, format="%m/%d/%Y")
weather["month"] = weather["date"].apply(lambda x: x.month)
weather["day"] = weather["date"].apply(lambda x: x.day)
weather["year"] = weather["date"].apply(lambda x: x.year)
weather['weekday'] = weather['date'].apply(lambda x: x.weekday())
weather['DATE'] = pd.to_datetime(weather[['year', 'month', 'day']], yearfirst=True)
weather["max_temperature_c"] = weather["max_temperature_f"].apply(lambda x: (x - 32) / 1.8)
weather["min_temperature_c"] = weather["min_temperature_f"].apply(lambda x: (x - 32) / 1.8)
weather["mean_temperature_c"] = weather["mean_temperature_f"].apply(lambda x: (x - 32) / 1.8)
weather["precipitation_inches"] = pd.to_numeric(weather["precipitation_inches"], errors="coerce")
weather["llueve"] = weather["events"].apply(lambda x: 0 if x != "rain" else 1)
weather["dias_lluvia"] = weather["precipitation_inches"].apply(lambda x: 1)
weather["n_events"] = weather["events"].apply(lambda x: func(x))

In [ ]:
labels = meses
index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
weatherbymonth = weather.groupby('month').mean()['mean_temperature_f']
weatherbymonth.plot(kind= 'bar')
plt.xticks(index, labels)
plt.title(" Temperatura promedio del dia por cada mes")
plt.xlabel("Meses")
plt.ylabel(' Temperatura promedio del dia [C] ')
plt.show()

In [ ]:
events = weather.groupby('n_events').aggregate(sum)['dias_lluvia']
events.plot(kind='bar')
plt.title('Cantidad de eventos meteorologicos')
index = [0, 1, 2, 3]
labels = ["Rain-Thunderstorm", "Fog", "Rain", "Fog-Rain"]
plt.xticks(index, labels)
plt.xlabel('Eventos')
plt.ylabel('Cantidad de dias que ocurrio')
plt.show()

In [ ]:
def combinar_trips_weather(lista_atributos):
    zip_list = []
    station_id_city = stations[['id', 'city']]
    station_id_city = station_id_city.rename(columns={'id': 'start_station_id'})
    trips_aux = trips.merge(station_id_city[['start_station_id', 'city']])
    viajes = trips_aux[['start_date', 'city','weekday']]

    for city in viajes.city:
        if (city == 'San Francisco'):
            zip_list.append([94107])
        elif (city == 'Redwood City'):
            zip_list.append([94063])
        elif (city == 'Mountain View'):
            zip_list.append([94041])
        elif (city == 'San Jose'):
            zip_list.append([95113])
        elif (city == 'Palo Alto'):
            zip_list.append([94301])

    zip_code_Df = pd.DataFrame(zip_list, columns={'zip_code'})
    trips_df= pd.concat([viajes , zip_code_Df], axis=1)
    trip_df = trips_df[['start_date', 'zip_code','weekday']]
    trips_df.insert(3, 'trips', 1)
    trips_df = trips_df.groupby(['start_date', 'zip_code']).aggregate(sum).reset_index()
    trips_df.start_date = pd.to_datetime(trips_df.start_date)
    trips_df = trips_df.rename(columns={'start_date': 'date'})
    weather_aux = weather
    weather_aux.date = pd.to_datetime(weather_aux.date)
    lista_aux = ["date", "zip_code"]
    for elemento in lista_atributos:
        lista_aux.append(elemento)
    weather_aux = weather[lista_aux]
    return pd.merge(weather_aux, trips_df)

In [ ]:
trips_weather= combinar_trips_weather(["n_events", "precipitation_inches"])
trips_weather_by_events = trips_weather[trips_weather.n_events.isin([0,1,2,3])]
trips_weather_by_events.plot.scatter("n_events", "precipitation_inches", alpha=0.25, figsize=(12,8), s=trips_weather_by_events['trips'])
plt.title("Cantidad de Viajes durante eventos meteorologicos con determinada precipitacion")
index = [0, 1, 2, 3]
labels = ["Rain-Thunderstorm", "Fog", "Rain", "Fog-Rain"]
plt.xticks(index, labels)
plt.xlabel("Eventos")
plt.ylabel("Precipitation inches")
plt.show()